# LSIによる次元圧縮の動作確認

ストーリー: https://www.pivotaltracker.com/story/show/149341617

参考文献:
- https://abicky.net/2012/03/24/211818/
- http://scikit-learn.org/stable/modules/decomposition.html#truncated-singular-value-decomposition-and-latent-semantic-analysis

Gemsimを使うほうがもしかしたらいいかもしれない
- http://blog.yuku-t.com/entry/20110623/1308810518

今回は、sckit-learnのTruncatedSVDを用いて次元圧縮を行ってみる

In [1]:
import sys
import os

os.getcwd()

'/Users/shwld/project/mofmof/donusagi-bot/learning/prototype'

In [2]:
learning_dir = os.path.abspath("../")
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

In [3]:
from learning.core.nlang import Nlang
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

## データの準備

In [4]:
df = pd.read_csv('./fixtures/learning_training_messages/toyotsu_human.csv')
sentences = np.array(df['question'])
separated_sentences = Nlang.batch_split(sentences)
separated_sentences

['住宅 ローン 法人 提携 銀行 トーメン 社員 申請 内容 修正 する 差戻し 願 する',
 '住宅 ローン 法人 提携 銀行 トーメン 社員 当社 提携 する 銀行 名 優遇 条件 連絡 窓口 等 知る',
 '在職 証明 書 欲しい',
 '住宅 ローン 手続き 法人 提携 銀行 トーメン 社員 不動産 法人 割引 紹介 カード 欲しい',
 '財形 貯蓄 財形 住宅 新規 募集 時期 いる',
 '財形 貯蓄 財形 住宅 金額 変更 随時 できる',
 '財形 貯蓄 財形 住宅 払戻し 願 する',
 '社宅 どこ 空き 状況',
 '出張 者 金町 寮 宿泊 する どう する いい',
 '引越 先 決まる ない どう する いい',
 '港 区 勤務 する 在勤 証明 書 発行 する ほしい',
 '人事 異動 出向 戻る 伴う ＰＣ アドレス 設定 どう やる',
 '育児 短時間 勤務 申請 書 いつ 提出 する',
 '育児 短時間 勤務 申請 書 曜日 別 申請 できる',
 '育児 短時間 勤務 申請 どこ 提出 する よい',
 '財形 貯蓄 財形 住宅 一部 解約 する 申込 振込 どれ かかる',
 '現在 借上げ 寮 いる 子供 大学 入学 する 上京 する 一緒 住める 寮 変更 する',
 'コーポレート カード 持つ 解る ない なる 調べる ほしい',
 '芝浦 アイランド 付近 住む 都 バス 利用 する 通勤 費 申請 する よい',
 '通勤 費 申請 最寄駅 バス 走行 距離 １ ｋｍ １ ｋｍ 未満 支給 なる なぜ',
 '東京 本社 近い バス 通勤 申請 する よい',
 '通勤 費 経済 的 合理 的 ルート',
 '通勤 費 月額 １０ 万 円 超える ない 新幹線 通勤 認める',
 '海外 送付 制度 利用 する 現地 発生 関税 費用 等 立替 精算',
 '現地 住所 決まる ない 引越 手配 進める られる',
 '予防 接種 料金 負担 先 どこ 勘定 科目',
 '休暇 申請 申請 内容 変更 する 申請 する いい',
 '在中 健康 診断 予約 変更 する どう する いい',
 '健保 健 診る 半年 受診 する 渡航 健 診る 代用 できる',
 '駐在 予定 中国 指定 健 診る 

## TfidfVectorizerを用いてベクトルを作る

In [5]:
vectorizer = TfidfVectorizer(use_idf=True, token_pattern=u'(?u)\\b\\w+\\b')
features = vectorizer.fit_transform(separated_sentences)

## 検索する単語も同様に処理する

In [6]:
target_sentences = np.array(['保険証なくした'])
target_separated_sentences = Nlang.batch_split(target_sentences)
target_features = vectorizer.transform(target_separated_sentences)
print(target_separated_sentences)
target_sentences2 = np.array(['保険証失くした'])
target_separated_sentences2 = Nlang.batch_split(target_sentences2)
target_features2 = vectorizer.transform(target_separated_sentences2)
target_separated_sentences2

['保険証 ない する']


['保険証 失 くい']

## コサイン類似検索をかけてみる(LSIなし)

In [7]:
similarities = cosine_similarity(features, target_features)
similarities = similarities.flatten()
zipped_data = zip(similarities, separated_sentences, np.array(df['question']))
list(sorted(zipped_data, key=lambda x: x[0], reverse=True))[0:10]

[(0.53408321406997084, '健康 保険証 紛失 する', '健康保険証を紛失した'),
 (0.49964333210885781,
  '豊田通商 健保 任意 継続 する 保険証 いつ 届く 届く 保険証',
  '豊田通商健保を任意継続したが、保険証はいつ届くの？\u3000届くまでの保険証は？'),
 (0.48267399886826184, '保険証 盗む れる', '保険証が盗まれた'),
 (0.39798431033064546, '保険証 印字 薄い なる', '保険証の印字が薄くなった'),
 (0.36713042709999311,
  'すぐ 病院 行く 保険証 忘れる どう する いい',
  'すぐに病院へ行きたいが保険証を忘れた、どうすればいいの？'),
 (0.2541693169394203, 'わかる ない', 'わからない'),
 (0.2541693169394203, '特に ない', '特にない'),
 (0.22372542796563233,
  '国内 出張 規定 のる ない 確認 する 海外 出張 フライト 東京 発 東京 移動 する 泊 する 場合 国内 日当 申請 する ない 理解',
  '国内出張規定にものっていないので確認したいのですが\n海外出張のフライトが東京発であったため東京へ移動し、前泊した場合\n国内の日当は申請しないという理解であってるか？\n'),
 (0.20184168223730012, '引越 先 決まる ない どう する いい', '引越先が決まっていないが、どうしたらいいか。'),
 (0.18933582019033379,
  '海外 出張 申請 する 料金 負担 ない ０ 円 計算 書 提出 する',
  '海外出張申請したが、料金負担はなし（0円）でも、計算書は提出するの？')]

In [8]:
similarities = cosine_similarity(features, target_features2)
similarities = similarities.flatten()
zipped_data = zip(similarities, separated_sentences, np.array(df['question']))
list(sorted(zipped_data, key=lambda x: x[0], reverse=True))[0:10]

[(0.58960630289081473, '健康 保険証 紛失 する', '健康保険証を紛失した'),
 (0.58921785745677258, '保険証 盗む れる', '保険証が盗まれた'),
 (0.57929402755271386,
  '豊田通商 健保 任意 継続 する 保険証 いつ 届く 届く 保険証',
  '豊田通商健保を任意継続したが、保険証はいつ届くの？\u3000届くまでの保険証は？'),
 (0.48583404779265327, '保険証 印字 薄い なる', '保険証の印字が薄くなった'),
 (0.40529716736761873,
  'すぐ 病院 行く 保険証 忘れる どう する いい',
  'すぐに病院へ行きたいが保険証を忘れた、どうすればいいの？'),
 (0.0,
  '住宅 ローン 法人 提携 銀行 トーメン 社員 申請 内容 修正 する 差戻し 願 する',
  '住宅ローン（法人提携銀行・旧トーメン社員）申請内容を修正したいので、差戻しをお願したい'),
 (0.0,
  '住宅 ローン 法人 提携 銀行 トーメン 社員 当社 提携 する 銀行 名 優遇 条件 連絡 窓口 等 知る',
  '住宅ローン（法人提携銀行・旧トーメン社員）の当社と提携している銀行名、優遇条件、連絡窓口等を知りたい'),
 (0.0, '在職 証明 書 欲しい', '在職証明書が欲しい'),
 (0.0,
  '住宅 ローン 手続き 法人 提携 銀行 トーメン 社員 不動産 法人 割引 紹介 カード 欲しい',
  '住宅ローン手続き（法人提携銀行・旧トーメン社員）の不動産法人割引の紹介カードが欲しい。'),
 (0.0, '財形 貯蓄 財形 住宅 新規 募集 時期 いる', '財形貯蓄・財形住宅の新規募集時期はいつですか？')]

## TruncatedSVDで次元圧縮する

In [9]:
lsa = TruncatedSVD(n_components=583, algorithm='arpack')
lsa_features = lsa.fit_transform(features)
normalizer = Normalizer(copy=False)
lsa_features = normalizer.fit_transform(lsa_features)

In [10]:
lsa_target_features = lsa.transform(target_features)
lsa_target_features = normalizer.transform(lsa_target_features)
lsa_target_features2 = lsa.transform(target_features2)
lsa_target_features2 = normalizer.transform(lsa_target_features2)

In [11]:
# LSI無し
print(features.shape)
print(target_features.shape)
print(target_features2.shape)

# LSIあり
print(lsa_features.shape)
print(lsa_target_features.shape)
print(lsa_target_features2.shape)

(584, 1097)
(1, 1097)
(1, 1097)
(584, 583)
(1, 583)
(1, 583)


## コサイン類似検索をかけてみる(LSIあり)

In [12]:
lsa_similarities = cosine_similarity(lsa_features, lsa_target_features)
lsa_similarities = lsa_similarities.flatten()
zipped_data = zip(lsa_similarities, separated_sentences, np.array(df['question']))
list(sorted(zipped_data, key=lambda x: x[0], reverse=True))[:10]

[(0.54332749140837444, '健康 保険証 紛失 する', '健康保険証を紛失した'),
 (0.50829150042911053,
  '豊田通商 健保 任意 継続 する 保険証 いつ 届く 届く 保険証',
  '豊田通商健保を任意継続したが、保険証はいつ届くの？\u3000届くまでの保険証は？'),
 (0.49102845036950332, '保険証 盗む れる', '保険証が盗まれた'),
 (0.40487289481356459, '保険証 印字 薄い なる', '保険証の印字が薄くなった'),
 (0.37348497148197496,
  'すぐ 病院 行く 保険証 忘れる どう する いい',
  'すぐに病院へ行きたいが保険証を忘れた、どうすればいいの？'),
 (0.25856865321287403, 'わかる ない', 'わからない'),
 (0.25856865321287403, '特に ない', '特にない'),
 (0.22759782059899505,
  '国内 出張 規定 のる ない 確認 する 海外 出張 フライト 東京 発 東京 移動 する 泊 する 場合 国内 日当 申請 する ない 理解',
  '国内出張規定にものっていないので確認したいのですが\n海外出張のフライトが東京発であったため東京へ移動し、前泊した場合\n国内の日当は申請しないという理解であってるか？\n'),
 (0.20533529604109807, '引越 先 決まる ない どう する いい', '引越先が決まっていないが、どうしたらいいか。'),
 (0.19261297398551816,
  '海外 出張 申請 する 料金 負担 ない ０ 円 計算 書 提出 する',
  '海外出張申請したが、料金負担はなし（0円）でも、計算書は提出するの？')]

In [13]:
lsa_similarities = cosine_similarity(lsa_features, lsa_target_features2)
lsa_similarities = lsa_similarities.flatten()
zipped_data = zip(lsa_similarities, separated_sentences, np.array(df['question']))
list(sorted(zipped_data, key=lambda x: x[0], reverse=True))[:10]

[(0.60367507619306882, '健康 保険証 紛失 する', '健康保険証を紛失した'),
 (0.60327736194571013, '保険証 盗む れる', '保険証が盗まれた'),
 (0.59311673655197372,
  '豊田通商 健保 任意 継続 する 保険証 いつ 届く 届く 保険証',
  '豊田通商健保を任意継続したが、保険証はいつ届くの？\u3000届くまでの保険証は？'),
 (0.49742668010916608, '保険証 印字 薄い なる', '保険証の印字が薄くなった'),
 (0.41496808496090781,
  'すぐ 病院 行く 保険証 忘れる どう する いい',
  'すぐに病院へ行きたいが保険証を忘れた、どうすればいいの？'),
 (0.043536535074195484, '', 'またね'),
 (0.029390880010387553, '', 'ありません'),
 (4.1951981570595394e-16,
  '海外 出張 申請 ２つ 出張 間 １ 日 ない 出張 申請 ２ 件 作成 する',
  '海外出張申請について、２つの出張の間に1日しかない。それでも出張申請は２件作成すべきか？'),
 (3.701575620325745e-16, 'おはよう', 'おはようございます'),
 (3.5094845757736827e-16, '通勤 費 申告 いつ 申請 する いい', '通勤費申告はいつまでに申請すればいいか')]

## パラメータ調整

http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html

を参考にパラメータを調整してみた。

- n_components: 
  - 推奨されている100だと意図した結果にならない(盗まれるが一致してしまう)、この傾向は、400くらいまで同じ、「失くす」と「盗まれる」が「失くす」と「紛失する」よりも近いカテゴリになってしまうようだ。この値を近づけていくと、コサイン類似のみのときの結果に近づくので、featureの数くらいが良さそう(設定できる最大)
- algorithm:
  - よくわかっていないが、arpackのほうがsimilalityが高い気がする


# LDAも試してみる

試せそうだったので `LDA_in_scikit_learn_xx.ipynb` あたりを参考にLDAを試して比較してみた。

In [14]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_topics=584, 
                                max_iter=5,
                                learning_method='online')


lda_features = lda.fit_transform(features)
lda_features = normalizer.fit_transform(lda_features)

In [15]:
lda_target_features = lda.transform(target_features)
lda_target_features = normalizer.transform(lda_target_features)
lda_target_features2 = lda.transform(target_features2)
lda_target_features2 = normalizer.transform(lda_target_features2)

In [16]:
# LDA無し
print(features.shape)
print(target_features.shape)
print(target_features2.shape)

# LDAあり
print(lda_features.shape)
print(lda_target_features.shape)
print(lda_target_features2.shape)

(584, 1097)
(1, 1097)
(1, 1097)
(584, 584)
(1, 584)
(1, 584)


## コサイン類似検索をかけてみる(LDAあり)

In [17]:
lda_similarities = cosine_similarity(lda_features, lda_target_features)
lda_similarities = lda_similarities.flatten()
zipped_data = zip(lda_similarities, separated_sentences, np.array(df['question']))
list(sorted(zipped_data, key=lambda x: x[0], reverse=True))[:10]

[(0.99999849529189011, '今日 いい 天気', '今日はいい天気ですね'),
 (0.99999461468407291, '旅行 好き', '旅行は好きですか？'),
 (0.99999460987754341, '今日 天気', '今日のお天気は'),
 (0.99999460987754341, '今日 天気', '今日の天気は？'),
 (0.9999944299492366, '今日 予定', '今日の予定は？'),
 (0.99999431172756847, '猫 好き', '猫は好き'),
 (0.99999173805029262, '健康 保険証 紛失 する', '健康保険証を紛失した'),
 (0.99999074679523214, '特に ない', '特にない'),
 (0.99998626090971121, '指定 旅行 代理 店', '指定旅行代理店は？'),
 (0.99991846541412988,
  '駐在 予定 中国 指定 健 診る 受診 する 渡航 健 診る 受ける ない いい',
  '駐在予定ですが、中国指定の健診を受診したら、渡航前健診は受けなくてもいいですか？')]

## コサイン類似検索をかけてみる(LDAあり)

In [18]:
lda_similarities = cosine_similarity(lda_features, lda_target_features2)
lda_similarities = lda_similarities.flatten()
zipped_data = zip(lda_similarities, separated_sentences, np.array(df['question']))
list(sorted(zipped_data, key=lambda x: x[0], reverse=True))[:10]

[(0.99993911131599322, '特に ない', '特にない'),
 (0.99992842840039509, '猫 好き', '猫は好き'),
 (0.99992800591235032, '今日 予定', '今日の予定は？'),
 (0.99992735225126572, '今日 天気', '今日のお天気は'),
 (0.99992735225126572, '今日 天気', '今日の天気は？'),
 (0.9999273346326788, '旅行 好き', '旅行は好きですか？'),
 (0.99985427973438534, '今日 いい 天気', '今日はいい天気ですね'),
 (0.99981178508187196, '健康 保険証 紛失 する', '健康保険証を紛失した'),
 (0.9997882573249316, '指定 旅行 代理 店', '指定旅行代理店は？'),
 (0.99960502094601422,
  '駐在 予定 中国 指定 健 診る 受診 する 渡航 健 診る 受ける ない いい',
  '駐在予定ですが、中国指定の健診を受診したら、渡航前健診は受けなくてもいいですか？')]

パラメータ調整
http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html
を参考にパラメータを調整してみた。

何か間違っているのかもしれないが、色々調整してみても、上記のように全く予測できていない。